# Pipeline E field reconstruction with polarization

Plan
* Method and goals
* What we need
* Pipeline definition
* Status and conclusion


## 1) Method and goals

### 1.1) Some defintions

* Electric field $\vec{e}(t)$ of air shower is linear polarized : $\vec{e}(t)=e(t)\vec{p}$
* Polarization angle is defined as: 

$\alpha=(\vec{u_{\theta}},\vec{e})$

Where $\vec{u_{\theta}}$ tangential vector on sphere at $(\phi,\theta)$ direction along $\theta$ coordinnate, zenithal angle

* Tangential frame is  $(\vec{u_{\theta}},\vec{u_{\phi}}, \vec{n})$
* Direction of polarization is 

$\vec{p}=\cos(\alpha)\vec{u_{\theta}}+\sin(\alpha)\vec{u_{\phi}}$

* Polar frame is $(\vec{p},\vec{n}\wedge\vec{p}, \vec{n})$
* Effective length in polar frame:

$L_{pol}(\phi,\theta,\alpha)=L_{\theta}(\phi,\theta)\cos(\alpha)+L_{\phi}(\phi,\theta)\sin(\alpha)$

where $L_{\theta}(\phi,\theta)$ and $L_{\phi}(\phi,\theta)$ is given by antenna model.

### 1.2) Model of measure in polar frame

For antenna $i$, the model of measure is a "full" 1D equation in fourier space

$V^i=L_{pol}^i.RF^i.E_{pol} + G^i.RF^i$ 

Where:
* $V^i$ voltage see by ADC
* $G^i$ is galactic signal at the antenna $i$ output
* $RF^i$ RF chain of antenna $i$

It's like convolution equation with noise:

$v^i=h^i*e_{pol}+n^i$

### 1.3) Polar Wierner estimation

The classic method to solve this convolution problem in presence of noise is the Wiener deconvolution method which is a frequency filter, defined as

$\hat{E}_{pol}=W^i.V^i$

Wiener gain is defined as :

$W^i=\dfrac{H^{i*}.P_e}{|H^i|^2.P_e+P_{n^i}}$

Where:
* $P_e$ power spectrum density of electric field
* $P_n$ power spectrum density of noise

Each antenna axis provides an estimation $\hat{e}_{pol}$ if $||L^i_{eff}||>\epsilon$
### 1.4) Motivation and goals

* It is interesting to explore this way of reconstructing the electric field, introducing physics, the constraint of linear polarization, could help the resolution
* Frequency resolution seems well suited to our problem where certain frequencies are very noisy
* Moreover, in terms of computing capacity, it is very economical; there is no iteration, just quotient of complex vector and FFT
* But we have to manage 3 solutions
* We know polarization is constrained for a air shower, having the polarization angle as a parameter of the reconstruction could allow testing its consistency with physics.





## 2) What we need

For simulation and reconstruction we need Xmax position, antenna model, RFchain, galactic model and also


### 2.1) Model of polarization angle 

Defining the polarization angle as an adjustment parameter of the reconstruction seems to be a big challenge. I tried to do it this fall without success. Currently I changed strategy, the reconstruction is done considering that the electric field is produced by a air shower where the polarization angle is constrained. The polarization angle is estimated with the simulations with 5 parameters: distance, direction of xmax in the DU frame, direction of DU in the shower frame.

### 2.2) Model of power spectrum density of E field

Wiener deconvolution needs an estimation of power spectrum density of E field. 
From my observations the spectrum is almost affine with the log scale in the GRAND frequency band. When the electric field is strong the spectrum tends to be flat and steep when it is weak. 2 points of the spectrum are therefore sufficient. The model consists of sampling the value of these 2 points for the maximum voltage value and the xmax position on all triggered traces.

### 2.3) Polarization trigger with model of voltage 

For a given direction and polarization angle, the antenna transfer function will be the same. Given that the shower spectrum has a "log-affine" shape, if we assume that the slope change is small, then the spectra are proportional.  Another way to justify this is to point out that the electric field pulses all have the same shape up to a scale factor, the difference is the duration of the pulse. Since our detector is a linear time-invariant system, the voltage will also be proportional.
This should be verified in voltage simulations. If this is not the case, it is possible to create spectrum level intervals where the proportionality assumption is valid.

This allows us to define a proportionality constraint on the voltages measured for a air shower for a given direction.

### 2.4) Polarization trigger with reconstruction of E field

For this test you must have at least 2 antennas that are not blind, ie $L^i_{eff}{=}\llap{/\,}0$ in the direction of xmax. If the polar angle $\alpha_{pol}$ estimated is really the good one, the estimation of $\hat{e}_{pol}$ must close, including the maximum value. So this function:

$\delta(\alpha)=\displaystyle\sum_{i{=}\llap{/\,}j}|\hat{e}^i_{max}(\alpha)-\hat{e}^j_{max}(\alpha)|$

must have as local minimum $\alpha_{pol}$. For $\alpha$ in $[\alpha_{pol}-30°,\alpha_{pol}+30°]$ 


## 3) Pipeline definition

Unfortunately, I don't reproduce the DC2 data for voltage simulations. Evaluating the reconstruction method with simulations is a key point to determine whether it can be applied to real data. I decided to do my own simulations to master the correct definition of the transfer function.

### 3.1) Simulation pipeline

* Take from DC2.1rc2 tag of GRANDLIB
  * RF chain model
  * Default antenna model 
  * Galactic model
* 3D simulation like GRANDLIB

$V^i=\Bigr(\displaystyle\sum_{k=N,W,U}L_k^i(\phi,\theta).E_k + G^i \Bigl) .RF^i$

* All events are "near" of origin of GRAND frame, no coordinate transformation of Efield to adjust North variation
* Keep only trigger traces and store signal and noise separatly
* Keep only event with at least 5 traces
* Downslamping to 500 MHz
* Digitalize for ADC 14 bits
* Reduce size of traces before storage
* E field reference values
  * Polar angle
  * $t_{max}$ and $e_{max}$ for full frequency
  * $t_{max}$ and $e_{max}$ for GRAND band frequency
  * 2 points of power spectrum density

### 3.2) Data models

#### Polar angle

* for E field with max > 75 $\mu V/m$ compute
  * Direction of Xmax at DU level
  * Direction of DU in shower frame
  * Polar angle
* Store it in numpy file

#### PSD from voltage

With voltage simulated, compute:
* V max
* Direction of Xmax at DU level
* Polar angle
* Output model: 2 points of power spectrum density
* Store it in numpy file

#### Relative voltage and direction

With voltage simulated, compute:
* Direction of Xmax at DU level
* Polar angle
* Output model: relative V max, min for each axis, 6 values
* Store it in numpy file

### 3.3) Reconstruction pipeline

* With same model as simuation: antenna, RF chain
* Estimate polar angle with model polar
* Mode "no Wiener" simple division by transfert function
* Mode Wiener white noise with sigma
* Mode Wiener with PSD of noise
* Add zero-padding if necessary
* Return estimation in time space
* Comparaision of reference value $t_{max}$ and $e_{max}$

### 3.4) Trigger polar : TDB

#### with voltage TBD
* Define input parameter for data model relative voltage
* estimate relative V max, min for each axis, 6 values from model 
* define relative V max, min for each axis, 6 values from DU event 
* compute distance and apply threshold or define a probability ...

#### With Efield TBD
* Compute $\delta(\alpha)$ for $[\alpha_{pol}-30°,\alpha_{pol}+30°]$ 
* Polar angle of air shower is the local minimum ?

## 4) Status and conclusions

### 4.1) Polar angle model

First result with 1000 events is interesting. After a few plots of data model I realize that the polarization angle doesn't depend of zenith distance and mainly on the azimuth.

![polar](./polar_angle_phi.png)

The dispersion around sinusoid comes from  events where xmax is close. in fact we can find this property by calculating the polarization angle for a station located at the core position. Initially $\vec{e}$ is aligned with the Lorentz force:

$\vec{e}=\vec{v}\wedge\vec{B}$

and polar angle at DU level is $\alpha=(\vec{u_{\theta}},\vec{e})$
so 

$\cos(\alpha)=\vec{u_{\theta}}.(\vec{v}\wedge\vec{B})$

whith $\vec{v}$ is $-\vec{n}$ defined above, after simplication

$\alpha=\arccos(\cos\beta\sin\phi)$

where $\beta$ is the inclination of the magnetic field 61.6 degree for DC2 simulation. This relation is independant of $\theta$ (!) and match with DC2 data E field.

Error distribution of model with 200 DUs

![polar error](./polar_angle_error.png)


### 4.2) Simulation pipeline

All features are available. For E field dc2 file, 60% of events are rejected and 10% of traces are conserved

### 4.3) Reconstruction pipeline

#### No noise with fsample 2000MHz

Voltage without noise

![voltage](./Figure_6.png)


Polar E field in GRAND band

![voltage](./Figure_13.png)


Reconstruction

![voltage](./Figure_7.png)

The value of maximum isn't the same because the GRAND band frequency isn't really match with global transfer function of detector

The 3 estimation of E field are very close

![voltage](./Figure_9.png)

#### With galactic noise and no Weiner

Surprisingly it still works well for traces with good SNR

![voltage](./Figure_16.png)


![voltage](./Figure_18.png)


#### With noise with Weiner

Not yet tested, Wiener code is ready but not integrated ...

### 4.4) Conclusions

* The polar angle model already gives good results with 1000 events processed, we have 10 times more
* With a 3D simulation and a 1D reconstruction we obtain very good agreement in the noise-free case.
  * 3 estimation gives the same t_max and e_max
  * Good result also with 500MHz sampling
* Correct result with galactic noise whitout Wiener processing on the traces with high SNR (!)
* Now need to finish Wiener case and do statistic on reconctruction of t_max et e_max at different SNR